## This notebook is creating synthetic data with the temperature varying as well as the radius according the flame spreading model

### No emmission from the rest of the star and putting the hotspot close to the pole

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.offsetbox import AnchoredText

%matplotlib inline

from __future__ import print_function, division

import os
import sys
import math
import time


from matplotlib import rcParams
from matplotlib.ticker import MultipleLocator, AutoLocator, AutoMinorLocator
from matplotlib import gridspec
from matplotlib import cm
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches
from scipy.interpolate import interp1d

import xpsi

from xpsi import Parameter

from scipy.interpolate import Akima1DInterpolator


from xpsi.global_imports import _c, _G, _dpr, gravradius, _csq, _km, _2pi

/=============================================\
| X-PSI: X-ray Pulse Simulation and Inference |
|---------------------------------------------|
|                Version: 2.0.0               |
|---------------------------------------------|
|      https://xpsi-group.github.io/xpsi      |
\=============================================/

Imported GetDist version: 1.4
Imported nestcheck version: 0.2.1


In [2]:
xpsi.HotRegion.super_colatitude


AttributeError: type object 'HotRegion' has no attribute 'super_colatitude'

In [ ]:
plt.rcParams.update({"figure.facecolor": "lightgray",
                    "axes.facecolor": "lightgray"})

In [ ]:
start = time.time()

In [ ]:
def kev_kelvin(x):
    return np.log10(x*1000./(8.61733034*(10.**-5)))

def kelvin_kev(x):
    return (8.61733034*(10.**(x-5)))/1000.

In [ ]:
kev_kelvin(0.1)

In [ ]:
kev_kelvin(0.16)

In [ ]:
good_burst=np.array([ 2.,  3.,  4.,  5.,  7., 10., 11., 12., 13., 15., 16., 17., 18.,
       20., 21., 22., 24., 25., 26., 27.])


# Importing data for burst generic burst

generic_burst=np.loadtxt("../../generic_burst/Burst_count.txt")
generic_time=np.loadtxt("../../generic_burst/Burst_time.txt")

pulse_burst5=np.loadtxt("../../generic_burst/pulses/pulseburst_5.txt")

counts_burst5=np.loadtxt("../../generic_burst/counts/countsburst_5.txt")

generic_pulse=np.zeros((28, 16))
generic_counts=np.zeros((17,))
for k in good_burst:
    k=int(k)
    generic_pulse+=np.loadtxt("../../generic_burst/pulses/pulseburst_{}.txt".format(k))
    generic_counts+=np.loadtxt("../../generic_burst/counts/countsburst_{}.txt".format(k))
    


# Temperature
## Obtain from Zach's data

In [ ]:
# Importing temperature obtained form Zach

temp_zach=np.loadtxt("../../temperature.dat")
time_zach=np.loadtxt("../../time.dat")

z_burst=np.loadtxt("../../z_burst.dat")
z_time=np.loadtxt("../../z_time.da")

In [ ]:
plt.plot(z_time,z_burst)

In [ ]:


duration=150.
b_bin=0.25
l=int(duration/b_bin)

f=interp1d(time_zach, temp_zach)

tim=np.linspace(0,duration,l)

temperature=np.log10(f(tim))

In [ ]:
ls ../../

In [ ]:
# # Generating see and saving for future use
# from random import randint
# seed=np.array(())
# for i in range(l):
#     seed=np.append(seed,randint(0, 100000))
#     #print(i)
# np.savetxt("../../seed/seed1.dat",seed)

In [ ]:
#seed=np.loadtxt("../../seed/seed1.dat")
seed=np.loadtxt("../../seed/seed1.dat")

In [ ]:
#np.savetxt("time_bins",tim)

In [ ]:

plt.plot(tim, seed,"o")

In [ ]:


temperature_kev=kelvin_kev(temperature)

fig, ax=plt.subplots(1,2,figsize=(15,5))
ax[0].plot(tim, temperature_kev,"k")
ax[1].plot(tim, temperature,"k")
ax[0].set_xlabel("Time (s)")
ax[0].set_ylabel("Temperature (keV)")
ax[1].set_xlabel("Time (s)")
ax[1].set_ylabel("Temperature (kelvin)")
#ax[1].set_yscale(log")
#plt.savefig("images/Temperature_case1.png")

In [ ]:
fig,ax=plt.subplots(1,1)
ax.plot(tim, temperature_kev,"k")
#plt.plot(tim, temperature,"k")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Temperature (keV)")
#plt.set_xlabel("Time (s)")
anchored_text1 = AnchoredText("Case1",loc=1)
#ax[1].add_artist(anchored_text2)
ax.add_artist(anchored_text1)

#plt.savefig("images/Temperature_case1.pdf")

In [ ]:
#This temperature profile is the GS-1826 temperature profile

In [ ]:

def fit_sin(x, y):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega", "phase", "offset", "freq", "period" and "fitfunc"'''

    ###### Importing parameters
    
    ff = np.fft.fftfreq(len(x), (x[1]-x[0]))   # assume uniform spacing
    Fyy = abs(np.fft.fft(y))
    
    
    ##### Inititial guess
    
    guess_freq =abs(ff[np.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = np.std(y) * 2.**0.5
    guess_offset = np.mean(y)
    guess = np.array([guess_amp, 2.*np.pi*guess_freq, 0., guess_offset])
    
    #### Fitting now 

    def sinfunc(t, A, w, p, c):  
        return A * np.sin(w*t + p) + c
    
    
    popt, pcov = scipy.optimize.curve_fit(sinfunc, x, y, p0=guess)
    A, w, p, c = popt
    #print(popt)
    f = w/(2.*np.pi)
    fitfunc = lambda t: A * np.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": 1./f, "fitfunc": fitfunc, "maxcov": np.max(pcov), "rawres": (guess,popt,pcov)}


In [ ]:
def fit(t,a,b):
    return a+b*np.sin(t*resburst_generic["omega"]+resburst_generic["phase"])

def fitburst(t,a,b):
    return a+b*np.sin(t*resburst10["omega"]+resburst10["phase"])



In [ ]:
path="../../ST_modules/"
sys.path.append(path)

In [ ]:
rcParams['text.usetex'] = False
rcParams['font.size'] = 14.0

def veneer(x, y, axes, lw=1.0, length=8):
    """ Make the plots a little more aesthetically pleasing. """
    if x is not None:
        if x[1] is not None:
            axes.xaxis.set_major_locator(MultipleLocator(x[1]))
        if x[0] is not None:
            axes.xaxis.set_minor_locator(MultipleLocator(x[0]))
    else:
        axes.xaxis.set_major_locator(AutoLocator())
        axes.xaxis.set_minor_locator(AutoMinorLocator())

    if y is not None:
        if y[1] is not None:
            axes.yaxis.set_major_locator(MultipleLocator(y[1]))
        if y[0] is not None:
            axes.yaxis.set_minor_locator(MultipleLocator(y[0]))
    else:
        axes.yaxis.set_major_locator(AutoLocator())
        axes.yaxis.set_minor_locator(AutoMinorLocator())

    axes.tick_params(which='major', colors='black', length=length, width=lw)
    axes.tick_params(which='minor', colors='black', length=int(length/2), width=lw)
    plt.setp(axes.spines.values(), linewidth=lw, color='black')

def plot_one_pulse(pulse, x, label=r'Counts', cmap=cm.magma, vmin=None, vmax=None):
    """ Plot a pulse resolved over a single rotational cycle. """

    fig = plt.figure(figsize = (7,7))

    gs = gridspec.GridSpec(1, 2, width_ratios=[50,1])
    ax = plt.subplot(gs[0])
    ax_cb = plt.subplot(gs[1])

    profile = ax.pcolormesh(x,
                             _data.channels,
                             pulse,
                             vmin = vmin,
                             vmax = vmax,
                             cmap = cmap,
                             linewidth = 0,
                             rasterized = True)

    profile.set_edgecolor('face')

    ax.set_xlim([0.0, 1.0])
    #ax.set_ylim(0,50)
    #ax.set_yscale('log')
    ax.set_ylabel(r'Channel')
    ax.set_xlabel(r'Phase')

    cb = plt.colorbar(profile,
                      cax = ax_cb)

    cb.set_label(label=label, labelpad=25)
    cb.solids.set_edgecolor('face')

    veneer((0.05, 0.2), (None, None), ax)

    plt.subplots_adjust(wspace = 0.025)

## Custom Instrument

In [ ]:
a=1
b=30

In [ ]:
class CustomInstrument(xpsi.Instrument):
    """ A model of the RXTE-PCA telescope response. """

    def __call__(self, signal, *args):
        """ Overwrite base just to show it is possible.

        We loaded only a submatrix of the total instrument response
        matrix into memory, so here we can simplify the method in the
        base class.

        """
        matrix = self.construct_matrix()

        self._folded_signal = np.dot(matrix, signal)

        return self._folded_signal

    @classmethod
    def from_SWG(cls, matrix, edges, channel_edges=None,min_matrix=3,max_matrix=30):
        try:
             if channel_edges:
                channel_edges = np.loadtxt(channel_edges, dtype=np.double)
        except (OSError, IOError, TypeError, ValueError):
            print('channel_edges file could not be loaded.')
        RSP = np.loadtxt(matrix, dtype=np.double)
        RSP = RSP.transpose()
        edges = (np.loadtxt(edges, dtype=np.double))

        #Sub response matrice to select
        matrix=RSP[min_matrix:max_matrix,0:301]
        channels = np.arange(min_matrix, max_matrix)
        
        return cls(matrix, edges, channels, channel_edges[min_matrix:max_matrix+1,-2])


In [ ]:

# from Nicer_CustomInstrument import CustomInstrument

# Instrument = CustomInstrument.from_response_files(ARF = '../Settings/nicer_v1.01_arf.txt',
#                                              RMF = '../Settings/nicer_v1.01_rmf_matrix.txt',
#                                              max_input = 1500,
#                                              min_input = 0,
#                                              chan_min=a,
#                                              chan_max=b,
#                                              channel_edges = '../Settings/nicer_v1.01_rmf_energymap.txt')

#from CustomInstrument import CustomInstrument

Instrument = CustomInstrument.from_SWG(matrix="../../Instrument_settings/RSP_burst10.txt",
                                edges="../../Instrument_settings/energy_edges.txt",
                                channel_edges="../../Instrument_settings/Energymap.txt",
                                min_matrix=a,
                                max_matrix=b)


In [ ]:
class CustomInterstellar(xpsi.Interstellar):
    """ Apply interstellar attenuation. """

    def __init__(self, energies, attenuation, bounds, values = {}):

        assert len(energies) == len(attenuation), 'Array length mismatch.'

        self._lkp_energies = energies # for lookup
        self._lkp_attenuation = attenuation # for lookup

        N_H = Parameter('column_density',
                        strict_bounds = (0.0,10.0),
                        bounds = bounds.get('column_density', None),
                        doc = 'Units of 10^20 cm^-2.',
                        symbol = r'$N_{\rm H}$',
                        value = values.get('column_density', None))

        self._interpolator = Akima1DInterpolator(self._lkp_energies,
                                                 self._lkp_attenuation)
        self._interpolator.extrapolate = True

        super(CustomInterstellar, self).__init__(N_H)

    def attenuation(self, energies):
        """ Interpolate the attenuation coefficients.

        Useful for post-processing. 

        """
        return self._interpolate(energies)**(self['column_density']/0.4)

    def _interpolate(self, energies):
        """ Helper. """
        _att = self._interpolator(energies)
        _att[_att < 0.0] = 0.0
        return _att

    @classmethod
    def from_SWG(cls, path, **kwargs):
        """ Load attenuation file from the NICER SWG. """

        temp = np.loadtxt(path, dtype=np.double)

        energies = temp[:,0]

        attenuation = temp[:,2]

        return cls(energies, attenuation, **kwargs)


In [ ]:
interstellar = CustomInterstellar.from_SWG("../../ISM/ISM_frac.txt",
                                           bounds = dict(column_density = (0.0,10.0)))

## Custom Signal

In [ ]:
from xpsi.likelihoods.default_background_marginalisation import eval_marginal_likelihood
from xpsi.likelihoods.default_background_marginalisation import precomputation

class CustomSignal(xpsi.Signal):
    """

    A custom calculation of the logarithm of the likelihood.
    We extend the :class:`~xpsi.Signal.Signal` class to make it callable.
    We overwrite the body of the __call__ method. The docstring for the
    abstract method is copied.

    """

    def __init__(self, workspace_intervals = 1000, epsabs = 0, epsrel = 1.0e-8,
                 epsilon = 1.0e-3, sigmas = 10.0, support = None, **kwargs):
        """ Perform precomputation.

        :params ndarray[m,2] support:
            Prior support bounds for background count rate variables in the
            :math:`m` instrument channels, where the lower bounds must be zero
            or positive, and the upper bounds must be positive and greater than
            the lower bound. Alternatively, setting the an upper bounds as
            negative means the prior support is unbounded and the flat prior
            density functions per channel are improper. If ``None``, the lower-
            bound of the support for each channel is zero but the prior is
            unbounded.

        """

        super(CustomSignal, self).__init__(**kwargs)

        try:
            self._precomp = precomputation(self._data.counts.astype(np.int32))
        except AttributeError:
            print('Warning: No data... can synthesise data but cannot evaluate a '
                  'likelihood function.')
        else:
            self._workspace_intervals = workspace_intervals
            self._epsabs = epsabs
            self._epsrel = epsrel
            self._epsilon = epsilon
            self._sigmas = sigmas

            if support is not None:
                self._support = support
            else:
                self._support = -1.0 * np.ones((self._data.counts.shape[0],2))
                self._support[:,0] = 0.0

    def __call__(self, *args, **kwargs):
        self.loglikelihood, self.expected_counts, self.background_signal = \
                eval_marginal_likelihood(self._data.exposure_time,
                                          self._data.phases,
                                          self._data.counts,
                                          self._signals,
                                          self._phases,
                                          self._shifts,
                                          self._precomp,
                                          self._support,
                                          self._workspace_intervals,
                                          self._epsabs,
                                          self._epsrel,
                                          self._epsilon,
                                          self._sigmas,
                                          kwargs.get('llzero'),
                                          allow_negative=False)

In [ ]:
spacetime = xpsi.Spacetime.fixed_spin(314.0)

In [ ]:
bounds = dict(distance = (0.1, 1.0),                     # (Earth) distance
                mass = (1.0, 3.0),                       # mass
                radius = (3.0 * gravradius(1.0), 16.0),  # equatorial radius
                cos_inclination = (0.0, 1.0))      # (Earth) inclination to rotation axis

spacetime = xpsi.Spacetime(bounds=bounds, values=dict(frequency=314.0))


In [ ]:
bounds = dict(super_colatitude = (None, None),
              super_radius = (None, None),
              phase_shift = (-0.25, 0.75),
              super_temperature = (None, None))

# a simple circular, simply-connected spot
primary = xpsi.HotRegion(bounds=bounds,
                            values={}, # no initial values and no derived/fixed
                            symmetry=True,
                            omit=False,
                            cede=False,
                            concentric=False,
                            sqrt_num_cells=32,
                            min_sqrt_num_cells=10,
                            max_sqrt_num_cells=64,
                            num_leaves=100,
                            num_rays=200,
                            prefix='p') # unique prefix needed because >1 instance


In [ ]:
bounds = dict(super_colatitude = (None, None),
              super_radius = (None, None),
              phase_shift = (-0.25, 0.75),
              super_temperature = (None, None))

# a simple circular, simply-connected spot
hot = xpsi.HotRegion(bounds=bounds,
                            values={}, # no initial values and no derived/fixed
                            symmetry=True,
                            omit=False,
                            cede=False,
                            concentric=False,
                            sqrt_num_cells=32,
                            min_sqrt_num_cells=10,
                            max_sqrt_num_cells=64,
                            num_leaves=100,
                            num_rays=200,
                            is_antiphased=True, 
                            prefix='h') # unique prefix needed because >1 instance


In [ ]:
"""
###################################################################################################################
########################################       CustomPhotosphere for J1814-338      ###############################
########################################                  Analysis  (STU)           ###############################
########################################    Using numerical atmosphere for hot NS)  ###############################
###################################################################################################################
"""

from __future__ import print_function, division

import numpy as np
import math

import xpsi

class CustomPhotosphere(xpsi.Photosphere):
    """
    CustomPhotosphere made for bursting atmosphere like XTE J1814-338.

    A photosphere extension to preload the numerical  diluted atmosphere.

    Using hot diluted atmosphere computed by Siem .

    """

    @xpsi.Photosphere.hot_atmosphere.setter
    def hot_atmosphere(self, path):
        Siem = np.loadtxt(path, dtype=np.double)

        # This is to get the number of value computed

        E_range=len(set(Siem[:,0]))
        mu_range=len(set(Siem[:,1]))
        T_range=len(set(Siem[:,3]))
        g_range=len(set(Siem[:,4]))



#         logT = np.zeros(35)
#         logg = np.zeros(14)
#         mu = np.zeros(67)
#         logE = np.zeros(166)

        logT = np.zeros(T_range)
        logg = np.zeros(g_range)
        mu = np.zeros(mu_range)
        logE = np.zeros(E_range)


        reorder_buf = np.zeros((T_range,g_range,mu_range,E_range))

        index = 0
        for i in range(reorder_buf.shape[0]):
            for j in range(reorder_buf.shape[1]):
                for k in range(reorder_buf.shape[3]):
                    for l in range(reorder_buf.shape[2]):
                        logT[i] = Siem[index,3]
                        logg[j] = Siem[index,4]
                        logE[k] = Siem[index,0]
                        mu[reorder_buf.shape[2] - l - 1] = Siem[index,1]
                        reorder_buf[i,j,reorder_buf.shape[2] - l - 1,k] = 10.0**(Siem[index,2])
                        index += 1

        buf = np.zeros(np.prod(reorder_buf.shape))

        bufdex = 0
        for i in range(reorder_buf.shape[0]):
            for j in range(reorder_buf.shape[1]):
                for k in range(reorder_buf.shape[2]):
                    for l in range(reorder_buf.shape[3]):
                        buf[bufdex] = reorder_buf[i,j,k,l]; bufdex += 1

        self._hot_atmosphere = (logT, logg, mu, logE, buf)
        
        
    @xpsi.Photosphere.elsewhere_atmosphere.setter
    def elsewhere_atmosphere(self, path):
        Siem = np.loadtxt(path, dtype=np.double)

        # This is to get the number of value computed

        E_range=len(set(Siem[:,0]))
        mu_range=len(set(Siem[:,1]))
        T_range=len(set(Siem[:,3]))
        g_range=len(set(Siem[:,4]))



#         logT = np.zeros(35)
#         logg = np.zeros(14)
#         mu = np.zeros(67)
#         logE = np.zeros(166)

        logT = np.zeros(T_range)
        logg = np.zeros(g_range)
        mu = np.zeros(mu_range)
        logE = np.zeros(E_range)


        reorder_buf = np.zeros((T_range,g_range,mu_range,E_range))

        index = 0
        for i in range(reorder_buf.shape[0]):
            for j in range(reorder_buf.shape[1]):
                for k in range(reorder_buf.shape[3]):
                    for l in range(reorder_buf.shape[2]):
                        logT[i] = Siem[index,3]
                        logg[j] = Siem[index,4]
                        logE[k] = Siem[index,0]
                        mu[reorder_buf.shape[2] - l - 1] = Siem[index,1]
                        reorder_buf[i,j,reorder_buf.shape[2] - l - 1,k] = 10.0**(Siem[index,2])
                        index += 1

        buf = np.zeros(np.prod(reorder_buf.shape))

        bufdex = 0
        for i in range(reorder_buf.shape[0]):
            for j in range(reorder_buf.shape[1]):
                for k in range(reorder_buf.shape[2]):
                    for l in range(reorder_buf.shape[3]):
                        buf[bufdex] = reorder_buf[i,j,k,l]; bufdex += 1

        self._elsewhere_atmosphere = (logT, logg, mu, logE, buf)


    @property
    def global_variables(self):
        """ This method is needed if we also want to invoke the image-plane signal simulator.

        The extension module compiled is surface_radiation_field/archive/local_variables/two_spots.pyx,
        which replaces the contents of surface_radiation_field/local_variables.pyx.

        """
        return np.array([self['h__super_colatitude'],
                          self['h__phase_shift'] * _2pi,
                          self['h__super_radius'],
                          self['h__super_temperature']])#,
                          #self['s__super_colatitude'],
                          #(self['s__phase_shift'] + 0.5) * _2pi,
                          #self['s__super_radius'],
                          #self.hot.objects[1]['s__super_temperature']])


In [ ]:
# from __future__ import print_function, division

# import numpy as np
# import math

# import xpsi

# class CustomPhotosphere(xpsi.Photosphere):
#     """ Implement method for imaging."""

#     @property
#     def global_variables(self):

#         return np.array([self['h__super_colatitude'],
#                           self['h__phase_shift'] * _2pi,
#                           self['h__super_radius'],
#                           self['h__super_temperature']])


In [ ]:
bounds=dict(elsewhere_temperature = (5, 8))
elsewhere=xpsi.Elsewhere(bounds=bounds)

In [ ]:
photosphere = CustomPhotosphere(hot = hot, elsewhere = None,#elsewhere,
                                values=dict(mode_frequency = spacetime['frequency']))


photosphere.hot_atmosphere="../../Atmosph/solarabundance.txt"

In [ ]:
star = xpsi.Star(spacetime = spacetime, photospheres = photosphere)

In [ ]:

from scipy.stats import truncnorm



In [ ]:
class CustomPrior(xpsi.Prior):
    """ A custom (joint) prior distribution.

    Source: Fictitious
    Model variant: ST-U
        Two single-temperature, simply-connected circular hot regions with
        unshared parameters.

    """

    __derived_names__ = ['compactness', 'phase_separation',]

    def __init__(self):
        """ Nothing to be done.

        A direct reference to the spacetime object could be put here
        for use in __call__:

        .. code-block::

            self.spacetime = ref

        Instead we get a reference to the spacetime object through the
        a reference to a likelihood object which encapsulates a
        reference to the spacetime object.

        """
        super(CustomPrior, self).__init__() # not strictly required if no hyperparameters

    def __call__(self, p = None):
        """ Evaluate distribution at ``p``.

        :param list p: Model parameter values.

        :returns: Logarithm of the distribution evaluated at ``p``.

        """
        temp = super(CustomPrior, self).__call__(p)
        if not np.isfinite(temp):
            return temp

        # based on contemporary EOS theory
        if not self.parameters['radius'] <= 16.0:
            return -np.inf

        ref = self.parameters.star.spacetime # shortcut

        # limit polar radius to try to exclude deflections >= \pi radians
        # due to oblateness this does not quite eliminate all configurations
        # with deflections >= \pi radians
        R_p = 1.0 + ref.epsilon * (-0.788 + 1.030 * ref.zeta)
        if R_p < 1.76 / ref.R_r_s:
            return -np.inf

        # polar radius at photon sphere for ~static star (static ambient spacetime)
        #if R_p < 1.5 / ref.R_r_s:
        #    return -np.inf

        mu = math.sqrt(-1.0 / (3.0 * ref.epsilon * (-0.788 + 1.030 * ref.zeta)))

        # 2-surface cross-section have a single maximum in |z|
        # i.e., an elliptical surface; minor effect on support, if any,
        # for high spin frequenies
        if mu < 1.0:
            return -np.inf

        ref = self.parameters # redefine shortcut

#         # enforce order in hot region colatitude
#         if ref['p__super_colatitude'] > ref['s__super_colatitude']:
#             return -np.inf

#         phi = (ref['p__phase_shift'] - 0.5 - ref['s__phase_shift']) * _2pi

#         ang_sep = xpsi.HotRegion.psi(ref['s__super_colatitude'],
#                                      phi,
#                                      ref['p__super_colatitude'])

#         # hot regions cannot overlap
#         if ang_sep < ref['p__super_radius'] + ref['s__super_radius']:
#             return -np.inf

        return 0.0

    def inverse_sample(self, hypercube=None):
        """ Draw sample uniformly from the distribution via inverse sampling. """

        to_cache = self.parameters.vector

        if hypercube is None:
            hypercube = np.random.rand(len(self))

        # the base method is useful, so to avoid writing that code again:
        _ = super(CustomPrior, self).inverse_sample(hypercube)

        ref = self.parameters # shortcut

        idx = ref.index('distance')
        ref['distance'] = truncnorm.ppf(hypercube[idx], -2.0, 7.0, loc=0.3, scale=0.1)

        # flat priors in cosine of hot region centre colatitudes (isotropy)
        # support modified by no-overlap rejection condition
        idx = ref.index('p__super_colatitude')
        a, b = ref.get_param('p__super_colatitude').bounds
        a = math.cos(a); b = math.cos(b)
        ref['p__super_colatitude'] = math.acos(b + (a - b) * hypercube[idx])

#         idx = ref.index('s__super_colatitude')
#         a, b = ref.get_param('s__super_colatitude').bounds
#         a = math.cos(a); b = math.cos(b)
#         ref['s__super_colatitude'] = math.acos(b + (a - b) * hypercube[idx])

        # restore proper cache
        for parameter, cache in zip(ref, to_cache):
            parameter.cached = cache

        # it is important that we return the desired vector because it is
        # automatically written to disk by MultiNest and only by MultiNest
        return self.parameters.vector

    def transform(self, p, **kwargs):
        """ A transformation for post-processing. """

        p = list(p) # copy

        # used ordered names and values
        ref = dict(zip(self.parameters.names, p))

        # compactness ratio M/R_eq
        p += [gravradius(ref['mass']) / ref['radius']]

#         # phase separation between hot regions
#         # first some temporary variables:
#         if ref['p__phase_shift'] < 0.0:
#             temp_p = ref['p__phase_shift'] + 1.0
#         else:
#             temp_p = ref['p__phase_shift']

#         temp_s = 0.5 + ref['s__phase_shift']

#         if temp_s > 1.0:
#             temp_s = temp_s - 1.0

#         # now append:
#         if temp_s >= temp_p:
#             p += [temp_s - temp_p]
#         else:
#             p += [1.0 - temp_p + temp_s]

        return p

In [ ]:
prior = CustomPrior()

In [ ]:
from scipy.integrate import quad
from xpsi.Interstellar import Interstellar
from xpsi.global_imports import _keV, _k_B
k_B_over_keV = _k_B / _keV


class CustomBackground(xpsi.Background):
    """  ############         The background injected to generate synthetic data      #########################
    

    param dict bounds:
                     Bounds of the powerlaw index and the blackbody temperature (in log 10)
                     
    param dict values:
                     Values of the powerlaw index and temperature
                     
    
    param obj interstellar:
                    If  ``None``, the background is assumed to be local to the telescope.
                    
                    If not ``None``, the background is assumed to be defined as it might look like at the star, therefore NH correction must be applied.
                     
    """
    

    def __init__(self, bounds, values, interstellar = None):

        # Powerlaw component
        doc = """
        Powerlaw spectral index.
        """
        index = xpsi.Parameter('powerlaw_index',
                                strict_bounds = (1, 4),
                                bounds = bounds.get('powerlaw_index', None),
                                doc = doc,
                                symbol = r'$\Gamma$',
                                value = values.get('powerlaw_index', None))

        # Blackbody component 
        doc = """
        Background black body temperature.
        """
        background_temperature = xpsi.Parameter('background_BB_temperature',
                                strict_bounds = (3, 10),
                                bounds = bounds.get('background_BB_temperature', None),
                                doc = doc,
                                symbol = r'$T_{BB}$',
                                value = values.get('background_BB_temperature', None))

        
        super(CustomBackground, self).__init__(index, background_temperature)
        
        # Making sure the interstellar object is form Interstall class
        if interstellar is not None:
            if not isinstance(interstellar, Interstellar):
                raise TypeError('Invalid type for an interstellar object.')
            else:
                self._interstellar = interstellar
        else:
            self._interstellar = None


    def __call__(self, energy_edges, phases):
        """ Evaluate the incident background field. """

        G = self['powerlaw_index']
        T = self['background_BB_temperature']

        # Defining array that will be used later
        array_pl=np.array([])
        array_bb=np.array([])
        
        # KbT in keV
        temp=k_B_over_keV * pow(10.0, T)
        
        # Numerical intergration  for both PL and BB
        for i in range(len(energy_edges)-1):
            
            pl,_= quad(self.PowLaw, energy_edges[i], energy_edges[i+1], args=(G)) # Intergatting
            bb,_= quad(self.BlackBody, energy_edges[i], energy_edges[i+1], args=(temp))
            #print(pl)
            array_pl=np.append(array_pl,pl)
            array_bb=np.append(array_bb,bb)
            
            
            
        ######## Applying Normalization  in unit of photons/KeV/cm^2/s^1 #######
        
        k_pl=3.32*10**(-2) # See Krauss et al.2005: https://arxiv.org/pdf/astro-ph/0503671.pdf
        k_bb=(1.6/0.8)**2  # See https://heasarc.gsfc.nasa.gov/docs/xanadu/xspec/manual/node137.html
        array_pl *=k_pl
        array_bb *=k_bb*(1.0344*10**(-3))
        
        
        PL = np.zeros((energy_edges.shape[0] - 1, phases.shape[0]))
        BB = np.zeros((energy_edges.shape[0] - 1, phases.shape[0]))
        
        for i in range(phases.shape[0]):
            PL[:,i] = array_pl
            BB[:,i] = array_bb
            
            
        bkg=PL+BB
        
        # Apply Interstellar if not None
        if self._interstellar is not None:
            self._energy_mids=(energy_edges[1:]+energy_edges[:-1])/2
            self._interstellar(self._energy_mids, bkg) # Bad coding right ? :)
            


        self._incident_background =bkg
        ################### Just to see individual component: Not usefull code ##############
        self.pl=PL
        self.bb=BB
# #         self.plnew=temp4
# #         self.bbnew=temp6
        
    def PowLaw(self,energ, gamma):
        """ Defining powerlaw function
        E^-gamma
    
        """
        #self.energ=energ
        #self.gamma=gamma
        pl=energ**(-gamma)
        return pl
    
    def BlackBody(self,energy,k):
        """ Defining Blackbody function using xspec model
        
        See :https://heasarc.gsfc.nasa.gov/docs/xanadu/xspec/manual/node137.html

        A(E)=E^2/(exp(E/kT)-1)

        """
        result=(energy**2)/(np.exp(energy/k)-1)
        #print(result)
        return result



In [ ]:
bounds = dict(powerlaw_index = (None, None),
              background_BB_temperature = (None, None))

background = CustomBackground(bounds=bounds,
                             values={},
                             interstellar=interstellar ) # use strict bounds, but do not fix/derive

In [ ]:
class SynthesiseData(xpsi.Data):
    """ Custom data container to enable synthesis. """

    def __init__(self, channels, phases, first, last):

        self.channels = channels
        self._phases = phases

        try:
            self._first = int(first)
            self._last = int(last)
        except TypeError:
            raise TypeError('The first and last channels must be integers.')
        if self._first >= self._last:
            raise ValueError('The first channel number must be lower than the '
                             'the last channel number.')

In [ ]:
_data = SynthesiseData(np.arange(a,b), np.linspace(0.0, 1.0, 17), 0, b-a-1 )

In [ ]:
from xpsi.tools.synthesise import synthesise_exposure as _synthesise


In [ ]:
def synthesise(self,
               exposure_time,
               expected_background_counts,
               #registered_background=back,
               name='synthetic',
               directory='./',
               **kwargs):
        """ Synthesise data set.

        """
        self._expected_counts, synthetic, bkg,_= _synthesise(exposure_time,
                                                             self._data.phases,
                                                             self._signals,
                                                             self._phases,
                                                             self._shifts,
                                                              expected_background_counts,
                                                             self._background.registered_background)
        
        #print(self._signals)
        #print()
        try:
            if not os.path.isdir(directory):
                os.mkdir(directory)
        except OSError:
            print('Cannot create write directory.')
            raise

        np.savetxt(os.path.join(directory, name+'_realisation.dat'),
                   synthetic,
                   fmt = '%u')
        
        np.savetxt(os.path.join(directory, name+'_expectedcounts.dat'),
                   self.expected_counts,
                   fmt = '%.8e')
        
        
        np.savetxt(os.path.join(directory, name+'photos_signal.dat'),
                   np.array(photosphere.signal[0][0]))




        self._write(self.expected_counts,
                    filename = os.path.join(directory, name+'_expected_hreadable.dat'),
                    fmt = '%.8e')

        self._write(synthetic,
                    filename = os.path.join(directory, name+'_realisation_hreadable.dat'),
                    fmt = '%u')

def _write(self, counts, filename, fmt):
        """ Write to file in human readable format. """

        rows = len(self._data.phases) - 1
        rows *= len(self._data.channels)

        phases = self._data.phases[:-1]
        array = np.zeros((rows, 3))

        for i in range(counts.shape[0]):
            for j in range(counts.shape[1]):
                array[i*len(phases) + j,:] = self._data.channels[i], phases[j], counts[i,j]

            np.savetxt(filename, array, fmt=['%u', '%.6f'] + [fmt])


In [ ]:
CustomSignal.synthesise = synthesise
CustomSignal._write = _write

In [ ]:
signal = CustomSignal(data = _data,
                        instrument = Instrument,
                        background = background,
                        interstellar = interstellar,
                        cache = True,
                        prefix='Instrument')



likelihood = xpsi.Likelihood(star = star, signals = signal,
                             num_energies=128,
                             threads=1,
                             externally_updated=False)


for h in hot.objects:
    h.set_phases(num_leaves = 100)


In [ ]:
likelihood

In [ ]:
%%capture
""" Here I'm creating some synthetic data with the temperature grid, from 1 to 7.039 KeV (shape 100 )
The exposure time for each data is 0.1 so that the total exposure time will be 100 s
"""
#for k in range(0,l):

#likelihood.externally_updated = True  

mass=2.1
Req=12.3
radius=.7

#Telse=7.
for k in range(len(temperature)):
    s=int(seed[k])
    %env GSL_RNG_SEED=$s
    p_T=[mass,
         Req,
         0.8, # 1.5 for 3 KeV and 5. for 7 KeV
         math.cos(15*np.pi/180),
         0.6,
         55*np.pi/180,
         radius,
         temperature[k],
         1.41,              # BKG
        7,               # BKG
        3                # NH
        ]

    Instrument_kwargs = dict(exposure_time=b_bin,
                             expected_background_counts=100.0,
                             name='new_synthetic_case1_{}_{}_{}'.format(b_bin,k,radius),
                             directory='./temp1.0.0/')

    likelihood.synthesise(p_T, force=True, Instrument=Instrument_kwargs) # SEED=0
    #print(p_T)
;

In [ ]:
print('Sampling took', (time.time()-start)/60, 'minutes')

In [ ]:
matrix_syn=np.zeros((29,16))
matrix_exp=np.zeros((29,16))
const_syn=[]
count_syn=[]
const_exp=[]
count_exp=[]

for k in range(0,l):
    syn=np.loadtxt('./temp1.0.0/new_synthetic_case1_{}_{}_{}_realisation.dat'.format(b_bin,k,radius))
    exp=np.loadtxt('./temp1.0.0/new_synthetic_case1_{}_{}_{}_expectedcounts.dat'.format(b_bin,k,radius))
    const_syn=np.append(const_syn,np.sum(syn))#-np.sum(b))
    count_syn=np.append(count_syn,np.sum(syn))
    matrix_syn +=syn
    const_exp=np.append(const_exp,np.sum(exp))#-np.sum(b))
    count_exp=np.append(count_exp,np.sum(exp))
    matrix_exp +=exp
    



# Calculate the weighted mean


## Photon flux:
$N_{T} \sim T^{3}$


## mean Temperature:

$T_{mean}=\sum_{i=0}^{n} \omega_{i}T_{i}$

$\omega_{i}=\frac{n_{i}}{N_{total}}$

In [ ]:
# Calculating the Tmean
coef=const_syn/np.sum(const_syn)
Tmean=np.log10(np.sum(coef*np.power(10,temperature)))


In [ ]:
coef.shape

In [ ]:
g=10**temperature

In [ ]:
np.log10(np.mean(g))

In [ ]:
#np.mean(T)

In [ ]:
Tmean=np.log10(np.average(10**temperature,weights=coef))

In [ ]:
Tmean

In [ ]:
#plot_one_pulse(matrix, _data.phases)


In [ ]:
matrix_syn.sum()/5000.

In [ ]:
n=int(matrix_syn.sum()/5000.)

In [ ]:
radius

In [ ]:
matrix_syn.sum()/n

In [ ]:
# I'm lazy so I'm trying to automate a=everything :)

#n=4               # How I decide to combine the burst so that the get the same number of counts
#m=matrix_syn.sum()/n   # relatif number of count per combined burst



cont=0
timing=np.array([])
for i in range(0,n-1):
    #print(i)
    globals()["M_syn{}".format(i)]=np.zeros([29,16])
    globals()["M_exp{}".format(i)]=np.zeros([29,16])
    #print(i)
    timing=np.append(timing,cont)
    #print(timing)
    while globals()["M_syn{}".format(i)].sum()<=5000.0:
        ##print(count)
        globals()["M_syn{}".format(i)] +=np.loadtxt('./temp1.0.0/new_synthetic_case1_{}_{}_{}_realisation.dat'.format(b_bin,cont,radius))
        globals()["M_exp{}".format(i)] +=np.loadtxt('./temp1.0.0/new_synthetic_case1_{}_{}_{}_expectedcounts.dat'.format(b_bin,cont,radius))
        #print(count,globals()["M{}".format(i)].sum())
        cont +=1
        #print(cont)
        
globals()["M_syn{}".format(n-1)]=np.zeros([29,16])
globals()["M_exp{}".format(n-1)]=np.zeros([29,16])
timing=np.append(timing,cont)
for i in range(cont,l):
    globals()["M_syn{}".format(n-1)] +=np.loadtxt('./temp1.0.0/new_synthetic_case1_{}_{}_{}_realisation.dat'.format(b_bin,i,radius))
    globals()["M_exp{}".format(n-1)] +=np.loadtxt('./temp1.0.0/new_synthetic_case1_{}_{}_{}_expectedcounts.dat'.format(b_bin,i,radius))
   
    cont +=1
    #print(cont)
timing=np.append(timing,cont)


In [ ]:
#M_syn16.sum()

In [ ]:
mid_time=b_bin*(timing[1:]+timing[0:-1])/2

In [ ]:
mid_time

In [ ]:
13.75+(-13.75+22.5)/2

In [ ]:
# a=np.zeros((29,16))
# for k in range(199,299):
#     a+=np.loadtxt('./temp/new_synthetic_case1_{}_{}_{}_realisation.dat'.format(b_bin,k,radius))

In [ ]:
timing*0.25/2

In [ ]:
#M_syn0.sum(),M_syn1.sum(),M_syn2.sum(),M_syn3.sum(),M_syn4.sum(),M_syn5.sum(),M_syn6.sum()

In [ ]:
exp_max=matrix_syn.sum(axis=0).max()
exp_min=matrix_syn.sum(axis=0).min()
alpha=(exp_max-exp_min)/2
c=exp_min+alpha
rms=alpha/(np.sqrt(2)*c)*100

In [ ]:
rms

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(9,7))
syn=ax.imshow(matrix_syn,aspect="auto",extent=[0,1,1,29], origin="lower", cmap=cm.magma)
ax.set_xlabel("Phases")
ax.set_ylabel("Channels")
anchored_text1 = AnchoredText("Case1",loc=1)
#ax[1].add_artist(anchored_text2)
ax.add_artist(anchored_text1)
plt.colorbar(syn,ax=ax)
#plt.savefig("images/PPM_case1.pdf")

In [ ]:
norm=count_syn.max()/z_burst.max()


In [ ]:
#x_z=np.linspace(0,)

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(15,5))
ti=np.linspace(0,duration,count_syn.shape[0])
ax[0].plot(ti, count_syn, "tomato",label="Synthetic burst")
ax[0].plot(ti, count_exp, "red", label="Expected burst")
ax[0].plot(z_time, norm*z_burst, "k:", label="GS-1826")
#ax[0].fill_between(ti, count_exp-np.sqrt(count_exp), count_exp+np.sqrt(count_exp), color="pink")
phase=np.linspace(0,1,16,endpoint=False)
ax[1].plot(phase, matrix_syn.sum(axis=0),"o",color="k", label="Synthetic burst")
ax[1].plot(phase, matrix_syn.sum(axis=0),":",color="k")
ax[1].plot(phase, matrix_exp.sum(axis=0), "darkred", label="Expected burst")
ax[1].fill_between(phase, matrix_exp.sum(axis=0)-np.sqrt(matrix_exp.sum(axis=0)), matrix_exp.sum(axis=0)+np.sqrt(matrix_exp.sum(axis=0)), color="pink")
ax[0].set_xlabel("Time (s)")
ax[0].set_ylabel("Total counts")
ax[1].set_xlabel("Phases")
ax[1].set_ylabel("Total counts")
anchored_text2 = AnchoredText("rms FA = "+str("%.1f" %rms)+"%",loc=8)
anchored_text1 = AnchoredText("Total counts: "+str(int(matrix_syn.sum())),loc=5)
ax[1].add_artist(anchored_text2)
ax[0].add_artist(anchored_text1)
ax[0].legend()
ax[1].legend()

#plt.savefig("images//light_curve_case1.pdf")
#plt.savefig("./generic_burst/hotspot_only/second/light_curve_case1.png")
plt.show()

In [ ]:
matrix_exp.sum()/150

In [ ]:
#generic_burst[50:].shape

In [ ]:
#plt.plot(generic_time[50:],(generic_burst[50:]-count[82:])**(0.25))

In [ ]:
# fig,ax=plt.subplots(1,2,figsize=(15,5))

# syn=ax[0].imshow(matrix,aspect="auto",extent=[0,1,1,30], origin="lower", cmap=cm.magma)
# ax[0].set_xlabel("Phases")
# ax[0].set_ylabel("Channels")
# ax[0].title.set_text('Synthetic burst')

# bur=ax[1].imshow(generic_pulse/good_burst.shape[0],aspect="auto",extent=[0,1,0,28], origin="lower", cmap=cm.magma)
# ax[1].set_xlabel("Phases")
# ax[1].set_ylabel("Channels")
# ax[1].title.set_text('Average burst')

# plt.colorbar(syn,ax=ax[0])
# plt.colorbar(bur,ax=ax[1])
# plt.savefig("generic_burst/hotspot_only/second/pulse_T_{}_{}_{}.png".format(tmin,tmax, radius))
# plt.savefig("generic_burst/hotspot_only/second/pulse_case1.png")

In [ ]:
for i in range(0,n):
    globals()["exp_max{}".format(i)]=globals()["M_exp{}".format(i)].sum(axis=0).max()
    globals()["exp_min{}".format(i)]=globals()["M_exp{}".format(i)].sum(axis=0).min()
    globals()["alpha{}".format(i)]=(globals()["exp_max{}".format(i)]-globals()["exp_min{}".format(i)])/2
    globals()["c{}".format(i)]=globals()["exp_min{}".format(i)]+globals()["alpha{}".format(i)]
    globals()["rms{}".format(i)]=globals()["alpha{}".format(i)]/(np.sqrt(2)*globals()["c{}".format(i)])*100

In [ ]:
for i in range(0,n):
    globals()["exp_max{}".format(i)]=globals()["M_exp{}".format(i)].sum(axis=0).max()
    globals()["exp_min{}".format(i)]=globals()["M_exp{}".format(i)].sum(axis=0).min()
    globals()["rms2{}".format(i)]=100*(globals()["exp_max{}".format(i)]-globals()["exp_min{}".format(i)])/(np.sqrt(2)*(globals()["exp_max{}".format(i)]+globals()["exp_min{}".format(i)]))

In [ ]:
# Calculation error on rms

for i in range(0,n):
    globals()["exp_max{}".format(i)]=globals()["M_exp{}".format(i)].sum(axis=0).max()
    globals()["exp_min{}".format(i)]=globals()["M_exp{}".format(i)].sum(axis=0).min()
    globals()["err_rms{}".format(i)]=np.sqrt((2*globals()["exp_max{}".format(i)]*globals()["exp_min{}".format(i)])/((globals()["exp_max{}".format(i)]+globals()["exp_min{}".format(i)])**3))

In [ ]:
m=int(np.ceil(n/3))

In [ ]:
fig,ax=plt.subplots(m,3, figsize=(15,20))
#for k in range(0,m):
for i in range(0,m):
    x_syn=np.linspace(0,1,16,endpoint=False)
    y_syn=globals()["M_syn{}".format(3*i)].sum(axis=0)
    x_exp=np.linspace(0,1,16,endpoint=False)
    y_exp=globals()["M_exp{}".format(3*i)].sum(axis=0)


    globals()["text{}".format(3*i)] = AnchoredText("% rms: "+str("%.2f" %(globals()["rms{}".format(3*i)])), loc=8)
    ax[i,0].plot(x_syn,y_syn,"o", color="k")
    ax[i,0].plot(x_syn,y_syn,":", color="k")
    ax[i,0].plot(x_exp,y_exp,"-", color="red")
    ax[i,0].fill_between(phase, y_exp-np.sqrt(y_exp), y_exp+np.sqrt(y_exp), color="pink")
    
    #globals()["text{}".format(3*i)] = AnchoredText(str(globals()["M_syn{}".format(3*i)].sum()), loc=1)
    ax[i,0].add_artist(globals()["text{}".format(3*i)])



    x_syn=np.linspace(0,1,16,endpoint=False)
    y_syn=globals()["M_syn{}".format(3*i+1)].sum(axis=0)
    x_exp=np.linspace(0,1,16,endpoint=False)
    y_exp=globals()["M_exp{}".format(3*i+1)].sum(axis=0)


    globals()["text{}".format(3*i+1)] = AnchoredText("% rms: "+str("%.2f" %(globals()["rms{}".format(3*i+1)])), loc=8)
    ax[i,1].plot(x_syn,y_syn,"o", color="k")
    ax[i,1].plot(x_syn,y_syn,":", color="k")
    ax[i,1].plot(x_exp,y_exp,"-", color="red")
    ax[i,1].fill_between(phase, y_exp-np.sqrt(y_exp), y_exp+np.sqrt(y_exp), color="pink")
    #globals()["text{}".format(3*i+1)] = AnchoredText(str(globals()["M_syn{}".format(3*i+1)].sum()), loc=1)
    ax[i,1].add_artist(globals()["text{}".format(3*i+1)])


    x_syn=np.linspace(0,1,16,endpoint=False)
    y_syn=globals()["M_syn{}".format(2*i+2)].sum(axis=0)
    x_exp=np.linspace(0,1,16,endpoint=False)
    y_exp=globals()["M_exp{}".format(2*i+2)].sum(axis=0)


    globals()["text{}".format(3*i+2)] = AnchoredText("% rms: "+str("%.2f" %(globals()["rms{}".format(3*i+2)])), loc=8)
    ax[i,2].plot(x_syn,y_syn,"o", color="k")
    ax[i,2].plot(x_syn,y_syn,":", color="k")
    ax[i,2].plot(x_exp,y_exp,"-", color="red")
    ax[i,2].fill_between(phase, y_exp-np.sqrt(y_exp), y_exp+np.sqrt(y_exp), color="pink")
    #globals()["text{}".format(3*i+2)] = AnchoredText(str(globals()["M_syn{}".format(3*i+2)].sum()), loc=1)
    ax[i,2].add_artist(globals()["text{}".format(3*i+2)])



    # ax[i].plot(xfit, fit(xfit,globals()["p{}".format(i)][0],globals()["p{}".format(i)][1]), ":", color="k")
   # ax[i].add_artist(globals()["text{}".format(i)])
    #ax[i].set_xlabel("Phases")
#     ax[0].set_ylabel("Counts")

    #globals()["textburst{}".format(i)] = AnchoredText("rms FA = "+str("%.1f" %(rms_burst[i]*100))+"%", loc=1)
    #ax[1,i].plot(x,globals()["bu{}".format(i)],"o", color="navy")
    #ax[1,i].plot(x, fitburst(x,globals()["pburst{}".format(i)][0],globals()["pburst{}".format(i)][1]), ":",color="navy")
    #ax[1,i].add_artist(globals()["textburst{}".format(i)])
    #ax[1,i].set_xlabel("Phases")

#ax[0].set_ylabel("Counts")
#ax[1,0].set_ylabel("Counts")
#plt.savefig("generic_burst/hotspot_only/second/rmsFA_-{}_{}_{}.png".format(tmin,tmax,radius))
#plt.savefig("generic_burst/hotspot_only/second/rmsFA_case1.png".format(tmin,tmax,radius))


In [ ]:
def fit(t,a,b):
    return a*t+b

r=[]
for i in range(0,n):
    r.append(globals()["rms{}".format(i)])
    
fa,fb=curve_fit(fit,mid_time,r)
xfit=np.linspace(0,150,150)

In [ ]:
def fit(t,a,b):
    return a*t+b

r=[]
for i in range(0,n):
    r.append(globals()["rms{}".format(i)])
    
fa,fb=curve_fit(fit,mid_time,r)
xfit=np.linspace(0,150,150)

f, ax = plt.subplots(1,1) 
for i in range(0,n):
    ax.plot(mid_time[i],globals()["rms{}".format(i)],"ko")
    #plt.errorbar(mid_time[i],globals()["rms{}".format(i)],yerr=globals()["err_rms{}".format(i)]*100,fmt="o",color='black',ecolor="k", elinewidth=None, capsize=4)

anchored_text1 = AnchoredText("Case1",loc=1)
#ax[1].add_artist(anchored_text2)
ax.add_artist(anchored_text1)

ax.plot(xfit,fit(xfit,fa[0],fa[1]), "k:")
ax.set_xlabel("Time (s)")
ax.set_xlim(0,150.)
ax.set_ylim(0,20.)
ax.set_ylabel("% rms")
#plt.savefig("images/rms_case1.pdf")


In [ ]:
#import os
#os.system('spd-say "Done"')

In [ ]:
#np.savetxt("../../data/data_case1_0_0.dat",matrix_syn)

In [ ]:
#plot_one_pulse(np.loadtxt("../../data/data_case1_0_0.dat"), _data.phases)

In [ ]:
#np.random.randint(1,15, size=21)

In [ ]:
#coef

In [ ]:
x=np.linspace(1,10000,10000000)
plt.plot(x,x*x+x*np.sqrt(x*x-0.1))